<a href="https://colab.research.google.com/github/FreMeli/ASE_1/blob/main/Regresion_ESCOLARIDAD_EDAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns #importamos la librería
sns.set()#inicializamos le estilo de los gráficos
import pandas as pd
import numpy as np
import pylab as pl
from pandas.plotting import scatter_matrix 
from matplotlib import cm 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import warnings

from google.colab import files 
import io

In [90]:
import pandas as pd
import numpy as np
import io
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
%matplotlib inline     
sns.set(color_codes=True)
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters.model import ExponentialSmoothing
import statsmodels.api as sm

In [4]:
upload = files.upload()

Saving INEGI_EDAD_ESCOLARIDAD.xls to INEGI_EDAD_ESCOLARIDAD.xls


In [79]:
df=pd.read_excel(io.BytesIO(upload["INEGI_EDAD_ESCOLARIDAD.xls"]), header=0)#df con los datos de la población
df

,AÑO,ENTIDAD,NIVEL DE ESCOLARIDAD,EDAD,HOMBRES,MUJERES,TOTAL
0,2000,Total,Sin escolaridad,Total,1118.0,170.0,1290
1,2000,Total,Sin escolaridad,5-9 años,8.0,NaN,8
2,2000,Total,Sin escolaridad,10-14 años,5.0,1.0,6
3,2000,Total,Sin escolaridad,15-19 años,34.0,2.0,36
4,2000,Total,Sin escolaridad,20-24 años,78.0,5.0,83
...,...,...,...,...,...,...,...
57670,2021,Zacatecas,Profesional,55-59 años,3.0,1.0,4
57671,2021,Zacatecas,Profesional,60-64 años,1.0,NaN,1
57672,2021,Zacatecas,Posgrado,Total,NaN,2.0,2
57673,2021,Zacatecas,Posgrado,30-34 años,NaN,1.0,1


In [80]:
df.isna().sum()

AÑO                         0
ENTIDAD                     0
NIVEL DE ESCOLARIDAD        0
EDAD                        0
HOMBRES                  2677
MUJERES                 29625
TOTAL                       0
dtype: int64

In [81]:
df.fillna(0, inplace=True)

In [82]:
df.isna().sum()

AÑO                     0
ENTIDAD                 0
NIVEL DE ESCOLARIDAD    0
EDAD                    0
HOMBRES                 0
MUJERES                 0
TOTAL                   0
dtype: int64

In [83]:
# Creamos una máscara booleana para seleccionar las filas que queremos eliminar
mask = (df['ENTIDAD'] == 'Total') | (df['ENTIDAD'] == 'No especificado') | (df['EDAD'] == 'Total')

# Aplicamos el método drop() para eliminar las filas seleccionadas por la máscara
df = df.drop(df[mask].index)
df

,AÑO,ENTIDAD,NIVEL DE ESCOLARIDAD,EDAD,HOMBRES,MUJERES,TOTAL
106,2000,Aguascalientes,Primaria incompleta,20-24 años,1.0,0.0,1
107,2000,Aguascalientes,Primaria incompleta,40-44 años,1.0,0.0,1
108,2000,Aguascalientes,Primaria incompleta,65-69 años,1.0,0.0,1
109,2000,Aguascalientes,Primaria incompleta,No especificado,0.0,1.0,1
111,2000,Aguascalientes,Primaria completa,10-14 años,1.0,0.0,1
...,...,...,...,...,...,...,...
57669,2021,Zacatecas,Profesional,50-54 años,2.0,0.0,2
57670,2021,Zacatecas,Profesional,55-59 años,3.0,1.0,4
57671,2021,Zacatecas,Profesional,60-64 años,1.0,0.0,1
57673,2021,Zacatecas,Posgrado,30-34 años,0.0,1.0,1


In [86]:
df.columns

Index(['AÑO', 'HOMBRES', 'MUJERES', 'TOTAL', 'ENTIDAD_Aguascalientes',
       'ENTIDAD_Baja California', 'ENTIDAD_Baja California Sur',
       'ENTIDAD_Campeche', 'ENTIDAD_Chiapas', 'ENTIDAD_Chihuahua',
       'ENTIDAD_Ciudad de México', 'ENTIDAD_Coahuila de Zaragoza',
       'ENTIDAD_Colima', 'ENTIDAD_Durango', 'ENTIDAD_Guanajuato',
       'ENTIDAD_Guerrero', 'ENTIDAD_Hidalgo', 'ENTIDAD_Jalisco',
       'ENTIDAD_Michoacán de Ocampo', 'ENTIDAD_Morelos', 'ENTIDAD_México',
       'ENTIDAD_Nayarit', 'ENTIDAD_Nuevo León', 'ENTIDAD_Oaxaca',
       'ENTIDAD_Puebla', 'ENTIDAD_Querétaro', 'ENTIDAD_Quintana Roo',
       'ENTIDAD_San Luis Potosí', 'ENTIDAD_Sinaloa', 'ENTIDAD_Sonora',
       'ENTIDAD_Tabasco', 'ENTIDAD_Tamaulipas', 'ENTIDAD_Tlaxcala',
       'ENTIDAD_Veracruz de Ignacio de la Llave', 'ENTIDAD_Yucatán',
       'ENTIDAD_Zacatecas',
       'NIVEL DE ESCOLARIDAD_Bachillerato o preparatoria completo',
       'NIVEL DE ESCOLARIDAD_Bachillerato o preparatoria incompleto',
       'NIVEL 

In [85]:
df = pd.get_dummies(df,columns=['ENTIDAD','NIVEL DE ESCOLARIDAD','EDAD'],dtype=int)

In [88]:
df.columns = ['Años','Hombres','Mujeres','Total',
              'Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche',
              'Chiapas', 'Chihuahua', 'Ciudad de México', 'Coahuila de Zaragoza', 
              'Colima', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 
              'Michoacán de Ocampo', 'Morelos', 'México', 'Nayarit', 'Nuevo León', 
              'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San Luis Potosí', 'Sinaloa',
              'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz de Ignacio de la Llave', 
              'Yucatán', 'Zacatecas','NIVEL DE ESCOLARIDAD_Bachillerato o preparatoria completo',
       'NIVEL DE ESCOLARIDAD_Bachillerato o preparatoria incompleto',
       'NIVEL DE ESCOLARIDAD_Posgrado', 'NIVEL DE ESCOLARIDAD_Preescolar',
       'NIVEL DE ESCOLARIDAD_Primaria completa',
       'NIVEL DE ESCOLARIDAD_Primaria incompleta',
       'NIVEL DE ESCOLARIDAD_Profesional',
       'NIVEL DE ESCOLARIDAD_Secundaria incompleta',
       'NIVEL DE ESCOLARIDAD_Secundaria o equivalente',
       'NIVEL DE ESCOLARIDAD_Sin escolaridad', 'EDAD_1-4 años',
       'EDAD_10-14 años', 'EDAD_15-19 años', 'EDAD_20-24 años',
       'EDAD_25-29 años', 'EDAD_30-34 años', 'EDAD_35-39 años',
       'EDAD_40-44 años', 'EDAD_45-49 años', 'EDAD_5-9 años',
       'EDAD_50-54 años', 'EDAD_55-59 años', 'EDAD_60-64 años',
       'EDAD_65-69 años', 'EDAD_70-74 años', 'EDAD_75-79 años',
       'EDAD_80-84 años', 'EDAD_85 años y más', 'EDAD_No especificado']

In [89]:
df.head()

,Años,Hombres,Mujeres,Total,Aguascalientes,Baja California,Baja California Sur,Campeche,Chiapas,Chihuahua,...,EDAD_5-9 años,EDAD_50-54 años,EDAD_55-59 años,EDAD_60-64 años,EDAD_65-69 años,EDAD_70-74 años,EDAD_75-79 años,EDAD_80-84 años,EDAD_85 años y más,EDAD_No especificado
106,2000,1.0,0.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107,2000,1.0,0.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
108,2000,1.0,0.0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
109,2000,0.0,1.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
111,2000,1.0,0.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
Y = df.Mujeres
X = df.drop(['Aguascalientes','EDAD_No especificado', 'NIVEL DE ESCOLARIDAD_Preescolar', 'Total', 'Mujeres'],axis=1)
X = sm.add_constant(X)

In [97]:
regresion = sm.OLS(Y,X).fit()

In [98]:
regresion.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Mujeres   R-squared:                       0.609
Model:                            OLS   Adj. R-squared:                  0.609
Method:                 Least Squares   F-statistic:                     1276.
Date:                Thu, 20 Apr 2023   Prob (F-statistic):               0.00
Time:                        21:56:25   Log-Likelihood:                -73481.
No. Observations:               49139   AIC:                         1.471e+05
Df Residuals:                   49078   BIC:                         1.476e+05
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
const                                                         -11.7808      1.685     -6.992      0.000     -15.083      -8.479
Años                                                            0.0057      0.001      6.893      0.000       0.004       0.007
Hombres                                                         0.0974      0.000    216.923      0.000       0.096       0.098
Baja California                                                -0.0064      0.050     -0.128      0.899      -0.105       0.092
Baja California Sur                                             0.0178      0.060      0.298      0.765      -0.099       0.135
Campeche                                                        0.0004      0.057      0.006      0.995      -0.112       0.113
Chiapas                                                         0.0649      0.051      1.265      0.206      -0.036       0.165
Chihuahua                                                       0.0127      0.050      0.255      0.799      -0.085       0.111
Ciudad de México                                                0.1948      0.049      3.938      0.000       0.098       0.292
Coahuila de Zaragoza                                            0.0916      0.052      1.748      0.081      -0.011       0.194
Colima                                                          0.0296      0.053      0.555      0.579      -0.075       0.134
Durango                                                        -0.1218      0.052     -2.347      0.019      -0.224      -0.020
Guanajuato                                                      0.1989      0.050      3.957      0.000       0.100       0.297
Guerrero                                                       -0.0170      0.049     -0.343      0.731      -0.114       0.080
Hidalgo                                                         0.1200      0.053      2.278      0.023       0.017       0.223
Jalisco                                                         0.0202      0.050      0.407      0.684      -0.077       0.117
Michoacán de Ocampo                                            -0.0701      0.050     -1.410      0.159      -0.167       0.027
Morelos                                                        -0.0408      0.050     -0.812      0.417      -0.139       0.058
México                                                          1.0177      0.050     20.546      0.000       0.921       1.115
Nayarit                                                        -0.0404      0.052     -0.775      0.438      -0.143       0.062
Nuevo León                                                      0.0682      0.051      1.335      0.182      -0.032       0.168
Oaxaca     

In [99]:
X.head()

,const,Años,Hombres,Baja California,Baja California Sur,Campeche,Chiapas,Chihuahua,Ciudad de México,Coahuila de Zaragoza,...,EDAD_45-49 años,EDAD_5-9 años,EDAD_50-54 años,EDAD_55-59 años,EDAD_60-64 años,EDAD_65-69 años,EDAD_70-74 años,EDAD_75-79 años,EDAD_80-84 años,EDAD_85 años y más
106,1.0,2000,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107,1.0,2000,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
108,1.0,2000,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
109,1.0,2000,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111,1.0,2000,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
X.columns

Index(['const', 'Años', 'Hombres', 'Baja California', 'Baja California Sur',
       'Campeche', 'Chiapas', 'Chihuahua', 'Ciudad de México',
       'Coahuila de Zaragoza', 'Colima', 'Durango', 'Guanajuato', 'Guerrero',
       'Hidalgo', 'Jalisco', 'Michoacán de Ocampo', 'Morelos', 'México',
       'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro',
       'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco',
       'Tamaulipas', 'Tlaxcala', 'Veracruz de Ignacio de la Llave', 'Yucatán',
       'Zacatecas',
       'NIVEL DE ESCOLARIDAD_Bachillerato o preparatoria completo',
       'NIVEL DE ESCOLARIDAD_Bachillerato o preparatoria incompleto',
       'NIVEL DE ESCOLARIDAD_Posgrado',
       'NIVEL DE ESCOLARIDAD_Primaria completa',
       'NIVEL DE ESCOLARIDAD_Primaria incompleta',
       'NIVEL DE ESCOLARIDAD_Profesional',
       'NIVEL DE ESCOLARIDAD_Secundaria incompleta',
       'NIVEL DE ESCOLARIDAD_Secundaria o equivalente',
       'NIVEL DE ESCOLARIDAD_Sin escolari

In [101]:
len(X.columns)

61

In [108]:
np.round(regresion.predict([1,2020,3,1,0,0,0,0,0,0,0,0,0,
                   0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                   0,0,0,1,0,0,0,0,0,0,0,0,
                   0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])) #Constabte,año,pobreza,pobreza extrema,estados-1

array([1.])

In [112]:
(df.Mujeres)

106      0.0
107      0.0
108      0.0
109      1.0
111      0.0
        ... 
57669    0.0
57670    1.0
57671    0.0
57673    1.0
57674    1.0
Name: Mujeres, Length: 49139, dtype: float64